In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import datetime

# Подготовка
Создадим словарь с User-Agent, чтобы авито не ловил нас сразу же.

In [2]:
base_url = 'https://www.avito.ru' 
url = 'https://www.avito.ru/sankt-peterburg/kvartiry/prodam/studii-ASgBAQICAUSSA8YQAUDKCBT~WA?cd=1&s=104&p={}'
ua = UserAgent()
headers = {'User-Agent': str(ua.chrome)}

# Вытащим ссылки на квартиры
Для этого я взял div с товаром и нашел в нем первую ссылку. Если пытаться просто вытаскивать ссылки с определенным классом, в них попадают также и ссылки на агенства и магазины (в случае парсинга товаров).

In [3]:
r = requests.get(url.format(1), headers=headers)
soup = BeautifulSoup(r.text, features='lxml')
divs = soup.find_all('div', {'class':'item__line'})
links = [div.find_next('a') for div in divs]

In [4]:
links[0].get('href')

'/sankt-peterburg/kvartiry/studiya_13.9_m_24_et._1900026283'

Теперь, когда все работает, соберем это в функцию

In [5]:
def get_links(page):
    r = requests.get(url.format(page), headers=headers)
    if not r.ok:
        return []
    soup = BeautifulSoup(r.text, features='lxml')
    divs = soup.find_all('div', {'class':'item__line'})
    links = []
    if divs:
        links = [div.find_next('a') for div in divs]
        if links:
            links = [base_url + link.get('href') for link in links]
    return links

In [6]:
links = get_links(1)
len(links)

51

In [7]:
link = links[0]

# Получим номер последней страницы
Чтобы не вводить номер последней страницы вручную, будем получать его автоматически

In [8]:
def get_last_page(soup):
    next_button = soup.find('span', {'data-marker':'pagination-button/next'})
    last_page = next_button.find_previous_sibling('span')
    return int(last_page.text)

In [9]:
get_last_page(soup)

100

# Парсинг страницы
Для примера возьмем самую первую квартиру и начнем с ней работать. Большая часть кода, написанная для парсинга страницы квартиры, будет работать и для страницы товара.

In [10]:
page_url = links[0]
print(page_url)

r = requests.get(page_url, headers=headers)
soup = BeautifulSoup(r.text, features='lxml')

https://www.avito.ru/sankt-peterburg/kvartiry/studiya_13.9_m_24_et._1900026283


## Заголовок

In [11]:
title = soup.find('span', {'class':'title-info-title-text'}).text
title

'Студия, 13.9 м², 2/4 эт.'

## Цена

In [12]:
price = int(soup.find('span', {'class':'js-item-price'}).get('content'))
price

1845000

## Имя продавца

In [13]:
seller_name = soup.find('div', {'class':'seller-info-name js-seller-info-name'}).text.strip()
seller_name

'Арт-Строй Групп'

## Рейтинг продавца
Поскольку рейтинг - вещественное число, если его нет, удобно воткнуть вместо него ``np.nan``

In [14]:
seller_rating = soup.find('span', {'class':'seller-info-rating-score'})
seller_rating = seller_rating.text.strip().replace(',', '.') if seller_rating else np.nan
seller_rating = float(seller_rating)
seller_rating

nan

## Количество отзывов

In [15]:
seller_review = soup.find('span', {'class':'seller-info-rating-caption'})
seller_review = seller_review.text.strip() if seller_review else '0'
seller_review = int(seller_review.split()[0])
seller_review

0

## Частное лицо / Компания

In [16]:
seller_status = soup.find_all('div', {'class':'seller-info-value'})[1]
seller_status = seller_status.find_previous_sibling('div').text.strip()
seller_status

'Агентство'

## Количество просмотров

In [17]:
views = soup.find('div', {'class':'title-info-metadata-item title-info-metadata-views'}).text
views = int(views[:views.find('(')].strip())
views

2599

## Адрес

In [18]:
address = soup.find('span', {'class':'item-address__string'}).text.strip()
address

'Санкт-Петербург, Средний пр-т Васильевского острова, 70'

## Ближайшее метро

In [19]:
metro = soup.find('span', {'class':'item-address-georeferences-item__content'}).text.strip()
to_metro = soup.find('span', {'class':'item-address-georeferences-item__after'}).text.strip()
to_metro = ' '.join(to_metro.split())
print(metro, to_metro)

Василеостровская 900 м


## Описание товара

In [20]:
desc = soup.find('div', {'class':'item-description'}).text.strip()
desc

'ПРОДАЁТСЯ УЮТНАЯ СТУДИЯ ПО ЦЕНЕ КОМНАТЫ В САМОМ СЕРДЦЕ ПИТЕРА В историческом центре города, 10 минутах ходьбы от станции метро «Василеостровская» для комфортного проживания или выгодной сдачи в аренду В районе отлично развита инфраструктура: Достопримечательности и места для посещения Магазины Кафе, рестораны Фитнес Дет. сады, школы ОПИСАНИЕ СТУДИИ: + Современный дизайнерский ремонт с учетом Ваших пожеланий + Тихий двор + Кирпичный дом + Комфортная зона отдыха + ванная комната с сантехникой и душевой + Возможность устройства спального места на втором ярусе ПРЕИМУЩЕСТВА: Коммунальные платежи 1000-1500 руб./мес. Нотариальное оформление сделки, расчёт через банковскую ячейку. Полное юридическое сопровождение на всех этапах. При покупке студии для сдачи в аренду, наша управляющая компания возьмёт на себя поиск и заселение арендаторов, заботу о сохранности объекта, хозяйственные затраты. Спешите записаться на показ!'

## Время добавления

In [21]:
added_time = soup.find('div', {'class':'title-info-metadata-item-redesign'}).text.strip()
added_time

'сегодня в 21:43'

## Характеристики квартиры

In [22]:
item_params = soup.find('div', {'class': 'item-params'})
key_to_column = {
    'Этаж': 'floor',
    'Этажей в доме': 'max_floor',
    'Тип дома': 'house_type',
    'Количество комнат': 'rooms',
    'Общая площадь':'square',
    'Год постройки': 'year_built'
}
params = {}
split = item_params.text.strip().split('\n')
for param in split:
    key, value = param.split(':')
    key = key_to_column[key.strip()]
    if key in ['floor', 'max_floor', 'year_built']:
        value = int(value.strip())
    elif key == 'rooms':
        value = value.strip().lower()
        if value == 'студии' or value == 'студия':
            value = 0
        value = int(value)
    elif key == 'square':
        value = ''.join([c for c in value.strip() if c.isnumeric() or c == '.'])[:-1]
        value = float(value)
    else:
        value = value.strip()
    params[key] = value
print(params)

{'floor': 2, 'max_floor': 4, 'house_type': 'кирпичный', 'rooms': 0, 'square': 13.9, 'year_built': 1895}


Теперь опять соберем все в одну функцию

In [34]:
def get_info(url, flat=False):
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, features='lxml')
    info = {}
    
    title = soup.find('span', {'class':'title-info-title-text'})
    if title:
        title = title.text
        info['title'] = title

    price = soup.find('span', {'class':'js-item-price'})
    if price:
        price = int(price.get('content'))
        info['price'] = price

    seller_name = soup.find('div', {'class':'seller-info-name js-seller-info-name'})
    if seller_name:
        seller_name = seller_name.text.strip()
        info['seller_name'] = seller_name
    
    seller_rating = soup.find('span', {'class':'seller-info-rating-score'})
    seller_rating = seller_rating.text.strip().replace(',', '.') if seller_rating else np.nan
    seller_rating = float(seller_rating)
    info['seller_rating'] = seller_rating
    
    seller_review = soup.find('span', {'class':'seller-info-rating-caption'})
    seller_review = seller_review.text.strip() if seller_review else '0'
    seller_review = int(seller_review.split()[0])
    info['seller_review'] = seller_review
    
    seller_status = soup.find_all('div', {'class':'seller-info-value'})
    if seller_status:
        seller_status = seller_status[1]
        seller_status = seller_status.find_previous_sibling('div').text.strip()
        info['seller_status'] = seller_status

    views = soup.find('div', {'class':'title-info-metadata-item title-info-metadata-views'})
    if views:
        views = views.text
        views = int(views[:views.find('(')].strip())
        info['views'] = views

    address = soup.find('span', {'class':'item-address__string'})
    if address:
        address = address.text.strip()
        info['address'] = address

    metro = soup.find('span', {'class':'item-address-georeferences-item__content'})
    metro = metro.text.strip() if metro else ''
    info['metro'] = metro
    
    to_metro = soup.find('span', {'class':'item-address-georeferences-item__after'})
    to_metro = to_metro.text.strip() if to_metro else ''
    to_metro = ' '.join(to_metro.split()) if to_metro else ''
    info['dist_to_metro'] = to_metro

    desc = soup.find('div', {'class':'item-description'})
    if desc:
        desc = desc.text.strip()
        info['description'] = desc

    added_time = soup.find('div', {'class':'title-info-metadata-item-redesign'})
    if added_time:
        added_time = added_time.text.strip()
        info['added_time'] = added_time
        
    if flat:
        item_params = soup.find('div', {'class': 'item-params'})
        key_to_column = {
            'Этаж': 'floor',
            'Этажей в доме': 'max_floor',
            'Тип дома': 'house_type',
            'Количество комнат': 'rooms',
            'Общая площадь':'square',
            'Жилая площадь': 'living_square',
            'Площадь кухни': 'kitchen_square',
            'Год постройки': 'year_built'
        }
        params = {}
        split = item_params.text.strip().split('\n')
        for param in split:
            key, value = param.split(':')
            key = key.strip()
            if key in key_to_column.keys():
                key = key_to_column[key.strip()]
            else:
                break
            if key in ['floor', 'max_floor', 'year_built']:
                value = int(value.strip())
            elif key == 'rooms':
                value = value.strip().lower()
                if value == 'студии' or value == 'студия':
                    value = 0
                value = int(value)
            elif 'square' in key:
                value = ''.join([c for c in value.strip() if c.isnumeric() or c == '.'])[:-1]
                value = float(value)
            else:
                value = value.strip()
            params[key] = value
            
        info.update(params)
    
    info['link'] = url
    info['parsed_at'] = datetime.now()

    return info

In [32]:
get_info(page_url, flat=True)

{'title': 'Студия, 13.9 м², 2/4 эт.',
 'price': 1845000,
 'seller_name': 'Арт-Строй Групп',
 'seller_rating': nan,
 'seller_review': 0,
 'seller_status': 'Агентство',
 'views': 2617,
 'address': 'Санкт-Петербург, Средний пр-т Васильевского острова, 70',
 'metro': 'Василеостровская',
 'dist_to_metro': '900 м',
 'description': 'ПРОДАЁТСЯ УЮТНАЯ СТУДИЯ ПО ЦЕНЕ КОМНАТЫ В САМОМ СЕРДЦЕ ПИТЕРА В историческом центре города, 10 минутах ходьбы от станции метро «Василеостровская» для комфортного проживания или выгодной сдачи в аренду В районе отлично развита инфраструктура: Достопримечательности и места для посещения Магазины Кафе, рестораны Фитнес Дет. сады, школы ОПИСАНИЕ СТУДИИ: + Современный дизайнерский ремонт с учетом Ваших пожеланий + Тихий двор + Кирпичный дом + Комфортная зона отдыха + ванная комната с сантехникой и душевой + Возможность устройства спального места на втором ярусе ПРЕИМУЩЕСТВА: Коммунальные платежи 1000-1500 руб./мес. Нотариальное оформление сделки, расчёт через банковску

# Протестируем, работает ли функция для парсинга товаров

In [25]:
page_url = 'https://www.avito.ru/sankt-peterburg/posuda_i_tovary_dlya_kuhni/ruchnaya_myasorubka_1900362626'
get_info(page_url)

{'title': 'Ручная мясорубка',
 'price': 800,
 'seller_name': 'Артем Рустамов',
 'seller_rating': nan,
 'seller_review': 0,
 'seller_status': 'Частное лицо',
 'views': 46,
 'address': 'Санкт-Петербург, Планерная ул., 63к1',
 'metro': 'Комендантский проспект',
 'dist_to_metro': '2,7 км',
 'description': 'PORKERT 10',
 'added_time': '28 марта в 16:06',
 'link': 'https://www.avito.ru/sankt-peterburg/posuda_i_tovary_dlya_kuhni/ruchnaya_myasorubka_1900362626',
 'parsed_at': datetime.datetime(2020, 4, 1, 22, 53, 34, 700285)}